In [1]:
from interest_similarity import interest_similarity
from get_recommended_items import get_recommended_items
from load_data import *
import warnings
warnings.filterwarnings("ignore")
# 노트북 1분 41초

FileNotFoundError: [Errno 2] No such file or directory: 'Data/whyout_data/user_interest.csv'

In [9]:
# user 3은 video만 있음, place, product 없음
# user 20001은 place만 있음, product, video 없음
# user 26232는 product만 있음, place, video 없음 

# 신규 유저는 관심항목 작성
user_id = 7
new_user_item = [1,0,1]
new_user_outdoor = [0,0,0,0,0,0,0,0,0,1]
num_recommendations = 10
case2_dict = { 'place' : [case2_sgd_rating_place, place, case2_user_place, case2_user_place_idx, case2_user_latent_place, case2_item_latent_place, user_place],
         'video' : [case2_sgd_rating_video, video, case2_user_video, case2_user_video_idx, case2_user_latent_video, case2_item_latent_video, user_video],
         'product' : [case2_sgd_rating_product, product, case2_user_product, case2_user_product_idx, case2_user_latent_product, case2_item_latent_product, user_product]}
item = 'product'
item_list = list(case2_dict.keys())

# Case 2만 구현
# if : 기존 유저, else : 신규 유저 구분하기
if user_id in user_interest['idx'].values:
    recomm_list = get_recommended_items(user_id, item, item_list, case2_dict, num_recommendations)
else:
    new_user_id = find_similar_user_interest(item, case2_dict, user_interest, new_user_item, new_user_outdoor)
    recomm_list = get_recommended_items(new_user_id, item, item_list, case2_dict, num_recommendations)
print(f"user {user_id}에게 추천해줄 {num_recommendations}개 {item} idx : {recomm_list}")

user 7에게 추천해줄 10개 product idx : [6089, 2741, 501, 4604, 2842, 3540, 1218, 3511, 1795, 4942]


In [1]:
from interest_similarity import interest_similarity
from get_recommended_items import get_recommended_items
import warnings
warnings.filterwarnings("ignore")

In [3]:
import pandas as pd
# user = pd.read_csv('../Data/whyout_data/user.csv') # (31177,3)

# user, place, product, video 메타데이터
user_interest = pd.read_csv('../Data/whyout_data/user_interest.csv') # (31177,4)
place = pd.read_csv('../Data/whyout_data/place.csv') # shape(4697,10)
product = pd.read_csv('../Data/whyout_data/product.csv') # shape(5821,11)
video = pd.read_csv('../Data/whyout_data/video.csv') # shape(3250, 9)

# [Case 1] 유저의 행동데이터 
user_place = pd.read_csv('../Data/whyout_data/user_place.csv') # (31177,4697)
user_product = pd.read_csv('../Data/whyout_data/user_product.csv') # (31177,5821)
user_video = pd.read_csv('../Data/whyout_data/user_video.csv') # (31177, 3250)

# [Case 2] 유저의 행동데이터 (행동 데이터가 없는 유저를 삭제한 데이터)
case2_user_place = pd.read_csv('../Data/whyout_data/case2_user_place.csv') # (22420,4697) 
case2_user_product = pd.read_csv('../Data/whyout_data/case2_user_product.csv') # (2994,5821)
case2_user_video = pd.read_csv('../Data/whyout_data/case2_user_video.csv') # (11067, 3250)

# [Case 2] 유저의 행동데이터 idx (행동 데이터가 없는 유저를 삭제한 데이터의 idx)
case2_user_place_idx = pd.read_csv('../Data/whyout_data/case2_user_place_idx.csv') # (22420,4)
case2_user_product_idx = pd.read_csv('../Data/whyout_data/case2_user_product_idx.csv') # (2294,4)
case2_user_video_idx = pd.read_csv('../Data/whyout_data/case2_user_video_idx.csv') # (11067, 4)

# [Case 2] SGD 결과(U x V) (행동 데이터가 없는 유저를 삭제한 데이터)
case2_sgd_rating_place = pd.read_csv('../Data/whyout_data/sgd_result/del_data/case2_sgd_rating_place.csv')
case2_sgd_rating_product = pd.read_csv('../Data/whyout_data/sgd_result/del_data/case2_sgd_rating_product.csv')
case2_sgd_rating_video = pd.read_csv('../Data/whyout_data/sgd_result/del_data/case2_sgd_rating_video.csv')

# [Case 2] SGD 결과의 user_latent(U) (행동 데이터가 없는 유저를 삭제한 데이터)item_latent(V)
case2_user_latent_place = pd.read_csv('../Data/whyout_data/sgd_result/del_data/case2_user_latent_place.csv')
case2_user_latent_product = pd.read_csv('../Data/whyout_data/sgd_result/del_data/case2_user_latent_product.csv')
case2_user_latent_video = pd.read_csv('../Data/whyout_data/sgd_result/del_data/case2_user_latent_video.csv')

# [Case 2] SGD 결과의 item_latent(V) (행동 데이터가 없는 유저를 삭제한 데이터)
case2_item_latent_place = pd.read_csv('../Data/whyout_data/sgd_result/del_data/case2_item_latent_place.csv')
case2_item_latent_product = pd.read_csv('../Data/whyout_data/sgd_result/del_data/case2_item_latent_product.csv')
case2_item_latent_video = pd.read_csv('../Data/whyout_data/sgd_result/del_data/case2_item_latent_video.csv')

In [43]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings("ignore")


def vectorize_outdoor(interests):
    if isinstance(interests, list):
        return interests
    else:
        return [0] * 10
def vectorize_item(interests):
    if isinstance(interests, list):
        return interests
    else:
        return [0] * 3

def make_user_interest(user_interest):
    df_filled = user_interest.fillna(0) # NaN 값을 0으로 대체

    # 관심 아이템, 아웃도어 컬럼의 문자열을 리스트로 변환
    df_filled['관심 아이템'] = df_filled['관심 아이템'].apply(lambda x: eval(x) if isinstance(x, str) else x)
    df_filled['관심 아웃도어'] = df_filled['관심 아웃도어'].apply(lambda x: eval(x) if isinstance(x, str) else x)
    
    # 관심 아이템, 아웃도어 0값 기존 차원과 동일하게 맞추기
    df_filled['관심 아이템'] = df_filled['관심 아이템'].apply(vectorize_item)
    df_filled['관심 아웃도어'] = df_filled['관심 아웃도어'].apply(vectorize_outdoor)

    df_features = df_filled.drop(columns=['idx', '나이']) # 불필요한 컬럼 제거
    df_features['Combined_Interest'] = df_features['관심 아이템'] + df_features['관심 아웃도어'] # 관심 아이템 + 관심 아웃도어를 하나의 차원으로 변경
    df_features = pd.DataFrame(df_features['Combined_Interest']) # 유사도 측정을 위한 데이터프레임 변환
    return df_features

def interest_similarity(item, case2_dict, user_interest, item_interest, outdoor_interest):
    df_features = make_user_interest(user_interest) # 기존 유저의 관심 아이템 + 관심 아웃도어의 전처리 작업
    new_data = item_interest + outdoor_interest # 새로운 유저의 관심 아이템 + 관심 아웃도어를 하나의 차원으로 변경
    print(new_data)
    # 신규 유저와 기존 유저의 관심 항목이 같은지 확인
    exact_match_indices = []
    for index, row in df_features.iterrows():
        if row['Combined_Interest'] == new_data:
            exact_match_indices.append(index)

    # 만약 기존 유저와 관심 항목이 일치하는게 없다면 if, 있으면 else
    if not exact_match_indices:
        combined_interest_matrix = np.array(df_features['Combined_Interest'].tolist()) # 코사인 유사도를 계산하기 위해 데이터프레임 변환
        cosine_sim_matrix = cosine_similarity([new_data], combined_interest_matrix) # 새로운 데이터와의 코사인 유사도 계산
        sorted_indices = np.argsort(-cosine_sim_matrix[0])
        sorted_similarity_scores = cosine_sim_matrix[0][sorted_indices]
        print(sorted_indices, sorted_similarity_scores)
        # 기존 아이템에 행동이 존재하는지 확인
        user_id, similarity_score = None, None
        for i, s in list(zip(sorted_indices, sorted_similarity_scores)): # (user_id의 index, 코사인 유사도 값)
            user_index = int(user_interest[user_interest.index == i].iloc[:,0]) # user_id의 idx(key)값 가져오기
            if user_index in case2_dict[item][3]['idx'].values: # key값이 원래 추천하려는 아이템에 존재한다면 if
                user_id = user_index
                similarity_score = s
                break
            else:
                print(f'유저 {user_index}가 {item}에 대한 행동이 없습니다.')
        print(f"{item}에 행동이 존재하며 유사도가 가장 높은 인덱스는 {user_id}이며, 유사도 점수는 {similarity_score}입니다.")
    else:
        # 관심항목이 정확히 일치하는 user_id에서 기존에 추천하려는 item에 행동이 있는지 확인
        match_idx = []
        for i in exact_match_indices: 
            i = int(user_interest[user_interest.index == i].iloc[:,0]) # user_id의 idx(key)값을 가져옴
            if i in case2_dict[item][3]['idx'].values: # 기존에 추천하려는 item에 행동이 있는지 확인
                match_idx.append(i) 
        print(f'{item}에 대한 행동이 있는 user index : {match_idx}')
        # 신규 유저와 관심항목이 동일한 기존 유저들 중 item에 대한 행동이 하나라도 존재하면
        if match_idx:
            user_actions = case2_dict[item][6].loc[match_idx].apply(lambda row : (row != 0).sum(), axis=1) # 행동이 있는 부분을 count함
            user_id = int(user_interest[user_interest.index == user_actions.idxmax()].iloc[:,0]) # 가장 행동이 많은 user_id를 뽑음
            print(f'{item}에 대한 행동이 가장 많은 유저 : user {user_id}')
        else:
            combined_interest_matrix = np.array(df_features['Combined_Interest'].tolist()) # 코사인 유사도를 계산하기 위해 데이터프레임 변환
            cosine_sim_matrix = cosine_similarity([new_data], combined_interest_matrix) # 새로운 데이터와의 코사인 유사도 계산
            sorted_indices = np.argsort(-cosine_sim_matrix[0])
            sorted_similarity_scores = cosine_sim_matrix[0][sorted_indices]

            # 기존 아이템에 행동이 존재하는지 확인
            user_id, similarity_score = None, None
            for i, s in list(zip(sorted_indices, sorted_similarity_scores)): # (user_id의 index, 코사인 유사도 값)
                user_index = int(user_interest[user_interest.index == i].iloc[:,0]) # user_id의 idx(key)값 가져오기
                if user_index in case2_dict[item][3]['idx'].values: # key값이 원래 추천하려는 아이템에 존재한다면 if
                    user_id = user_index
                    similarity_score = s
                    break
                else:
                    print(f'모든 유저가 {item}에 대한 행동이 없습니다.')
            print(f"{item}에 행동이 존재하며 유사도가 가장 높은 인덱스는 {user_id}이며, 유사도 점수는 {similarity_score}입니다.")
    return user_id

In [44]:
user_id = 38000
new_user_item = [1,0,0]
new_user_outdoor = [1,0,0,0,0,0,0,0,0,0]
num_recommendations = 10
case2_dict = { 'place' : [case2_sgd_rating_place, place, case2_user_place, case2_user_place_idx, 
                          case2_user_latent_place, case2_item_latent_place, user_place],
         'video' : [case2_sgd_rating_video, video, case2_user_video, case2_user_video_idx, 
                    case2_user_latent_video, case2_item_latent_video, user_video],
         'product' : [case2_sgd_rating_product, product, case2_user_product, case2_user_product_idx, 
                      case2_user_latent_product, case2_item_latent_product, user_product]}
item = 'video'
item_list = list(case2_dict.keys())

# Case 2만 구현
# if : 기존 유저, else : 신규 유저 구분하기
if user_id in user_interest['idx'].values:
    recomm_list = get_recommended_items(user_id, item, item_list, case2_dict, num_recommendations)
else:
    new_user_id = interest_similarity(item, case2_dict, user_interest, new_user_item, new_user_outdoor)
    recomm_list = get_recommended_items(new_user_id, item, item_list, case2_dict, num_recommendations)
print(f"user {user_id}에게 추천해줄 {num_recommendations}개 {item} idx : {recomm_list}")

[1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
video에 대한 행동이 있는 user index : [24, 48, 49, 165, 166, 585, 1005, 1070, 2054, 3200, 7168, 7542, 8007, 8069, 9921, 9995, 10621, 10928, 12871, 13478, 13844, 14505, 16331, 17724, 17867, 18242, 18391, 18740, 18811, 19016, 19168, 20635, 21648, 21861, 24592, 24731, 24732, 27324, 28299, 28603]
video에 대한 행동이 가장 많은 유저 : user 47
user 38000에게 추천해줄 10개 video idx : [3362, 85, 3188, 3327, 3476, 1586, 3557, 3215, 3473, 3132]
